# CSI4142 - Group 48 - Assignment 2

---

## Introduction
This report details data cleaning and imputation on 2 different datasets.

By presenting these findings through a combination of visualizations and actionable insights, this report aims to provide a comprehensive demonstration of data cleaning practices.

The target audience for this report includes ____.


#### Group 48 Members
- Ali Bhangu - 300234254
- Justin Wang - 300234186

In [109]:
# Importing the required Python libraries
import numpy as npy
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import os as os

In [110]:
# Checking to see if the csv exists, if so - delete and re-extract
csv_path = "dirty_cafe_sales.csv"
if os.path.exists(csv_path):
    print(f"Existing {csv_path} found. Deleting and re-extracting...")
    os.remove(csv_path)

#!/bin/bash
!curl -L -o ~/Downloads/cafe-sales-dirty-data-for-cleaning-training.zip\
  https://www.kaggle.com/api/v1/datasets/download/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training
  
# Check if the CSV file exists, otherwise raise an error
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"Dataset not found: {csv_path}. Ensure the file is downloaded and extracted.")

# Loading in the dataset
cafeSet = pd.read_csv(csv_path)
cafeSet.head()

Existing dirty_cafe_sales.csv found. Deleting and re-extracting...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  110k  100  110k    0     0   223k      0 --:--:-- --:--:-- --:--:-- 1844k


FileNotFoundError: Dataset not found: dirty_cafe_sales.csv. Ensure the file is downloaded and extracted.

In [ ]:
cafeSet = pd.read_csv("dirty_cafe_sales.csv")
cafeSet.info()


# cafeSet = cafeSet.infer_objects()
# print(cafeSet.dtypes)

# movieSet = pd.read_csv("movies.csv")
# movieSet.info()

netflixSet = pd.read_csv("netflix_titles.csv")
netflixSet.info()

newYorkSet = pd.read_csv("nyc-rolling-sales.csv")
newYorkSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null

## Data Type Error

A Data Type check ensures that data entered into a field is of the correct data type. A field, for example, may only accept numeric data. The system should then reject any data containing other characters, such as letters or special symbols, and an error message should be displayed.


### Parameters

In [ ]:
# Please enter the various attrivutes below to perform the data cleaning process on the dataset. 
# Input your column 
testColumn = "date_added"
expectedType = "object"

In [ ]:
from pandas.api.types import is_string_dtype, is_numeric_dtype

def check_data_type(df, column, expected_type):
      # Define a function to check if a value matches the expected type
    def is_expected_type(value):
        if pd.isna(value):
            return False  # Consider NaN as a mismatch
        return isinstance(value, expected_type)

    # Apply the function to the column and filter rows that don't match the expected type
    mismatched_rows = df[~df[column].apply(is_expected_type)]
    return mismatched_rows
    
# Example usage:
# Assuming 'netflixSet' is your DataFrame and 'date_added' is the column to check
errors = check_data_type(netflixSet, 'date_added', str)
if not errors.empty:
    print("Data type mismatch found in 'date_added' column:")
    print(errors)

## Range Error
A Range Check will determine whether the input data falls within a given range. Latitude and longitude, for example, are frequently used in geographic data. Latitude should be between -90 and 90, and longitude should be between -180 and 180. 

Any values outside of this range are considered invalid.


### Parameters

In [111]:
attributes = []

test_attribute = ""

minimum = 0
maximum = 0

In [ ]:
def range_checker(df, column, minimum, maximum):
    # Convert column to numeric, coerce errors to NaN
    numeric_col = pd.to_numeric(df[column], errors='coerce')
    
    # Filter directly using the numeric column while ignoring NaNs
    below_min = df.loc[(numeric_col < minimum) & numeric_col.notna()]
    above_max = df.loc[(numeric_col > maximum) & numeric_col.notna()]
    
    total_below = below_min.shape[0]
    total_above = above_max.shape[0]
    
    print(f"There are {total_below} data points with {column} less than {minimum}, and {total_above} data points with {column} over {maximum}.")
    
    if total_below > 0:
        print("\nExamples below minimum:")
        print(below_min.head(2))  # Show first 2 rows
    
    if total_above > 0:
        print("\nExamples above maximum:")
        print(above_max.head(2))  # Show first 2 rows

# Example usage:
range_checker(cafeSet, 'Price Per Unit', 3.0, 4.0)

## Format Errors

Many data types have a predefined format. A Format Check will ensure that the data is in the correct format. Date fields, for example, are stored in a fixed format such as “YYYY-MM-DD” or “DD-MM-YYYY.” If the date is entered in any other format, it will be rejected.


### Parameters

In [112]:
# Please enter the various attrivutes below to perform the data cleaning process on the dataset. 
# Input your column 
column = "Transaction Date"
pattern = r'^\d{4}-\d{2}-\d{2}$'

In [ ]:
import re

def format_checker(df, column, pattern):
    # Compile the regex pattern
    regex = re.compile(pattern)
    
    # Apply the regex pattern to the column and filter rows that don't match
    mismatched_rows = df[~df[column].astype(str).apply(lambda x: bool(regex.match(x)))]
    
    total_mismatched = mismatched_rows.shape[0]
    
    print(f"There are {total_mismatched} data points in {column} that do not match the format {pattern}.")
    
    if total_mismatched > 0:
        print("\nExamples of mismatched format:")
        print(mismatched_rows.head(2))  # Show first 2 rows

# Example usage:
format_checker(cafeSet, column, pattern)

There are 460 data points in Transaction Date that do not match the format ^\d{4}-\d{2}-\d{2}$.

Examples of mismatched format:
   Transaction ID      Item Quantity Price Per Unit Total Spent  \
11    TXN_3051279  Sandwich        2            4.0         8.0   
29    TXN_7640952      Cake        4            3.0        12.0   

    Payment Method  Location Transaction Date  
11     Credit Card  Takeaway            ERROR  
29  Digital Wallet  Takeaway            ERROR  


## Consistency Errors

A Consistency Check is a type of logical check that ensures data is entered in a logically consistent manner. Checking if the delivery date for a parcel is after the shipping date is one example



### Parameters

In [ ]:
# Please enter the various attrivutes below to perform the data cleaning process on the dataset. 

In [ ]:
import pandas as pd
import numpy as np

# pretty hard coded, could be improved with generalization

def consistency_checker(df):
    """
    Checks for consistency between Total Spent and Price Per Unit * Quantity.
    """
    # Convert to numeric, ignoring non-numeric values
    df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
    df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
    df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')
    
    # Calculate expected total
    df['expected_total'] = df['Price Per Unit'] * df['Quantity']
    
    # Find inconsistencies where Total Spent doesn't match expected total
    inconsistent = df[df['Total Spent'] != df['expected_total']]
    
    if inconsistent.empty:
        print("No consistency errors found.")
    else:
        print(f"Found {len(inconsistent)} consistency errors in 'Total Spent'.")
        print(inconsistent[['Transaction ID', 'Price Per Unit', 'Quantity', 'Total Spent', 'expected_total']].head(2))
    
    return inconsistent

# Example usage:
inconsistencies = consistency_checker(cafeSet)

Found 1456 consistency errors in 'Total Spent'.
   Transaction ID  Price Per Unit  Quantity  Total Spent  expected_total
2     TXN_4271903             1.0       4.0          NaN             4.0
20    TXN_3522028             4.0       NaN         20.0             NaN


## Uniqueness Errors

Some data, such as IDs or e-mail addresses, are inherently unique. These fields in a database should most likely have unique entries. A Uniqueness Check ensures that an item is not entered into a database more than once.

### Parameters

In [ ]:
testColumn = 'Transaction ID'

In [ ]:
# add user input firleds above ^ 
def uniqueness_checker(df, column):
    """
    Checks for uniqueness errors in a specified column.
    
    Parameters:
    df : DataFrame
    column : str, column name to check for unique values.
    
    Returns:
    DataFrame with duplicate rows.
    """
    duplicates = df[df.duplicated(subset=[column], keep=False)]
    
    if duplicates.empty:
        print(f"All values in '{column}' are unique.")
    else:
        print(f"Found {len(duplicates)} duplicate entries in '{column}'.")
        print(duplicates[[column]].head(2))  # Display first 2 duplicates
    
    return duplicates

# Example usage:
duplicates = uniqueness_checker(cafeSet, testColumn)

Found 2 duplicate entries in 'Transaction ID'.
  Transaction ID
0    TXN_1961373
1    TXN_1961373


## Presence Errors

A Presence Check ensures that all mandatory fields are not left blank. If someone tries to leave the field blank, an error message will be displayed, and they will be unable to proceed to the next step or save any other data that they have entered. A key field, for example, cannot be left blank in most databases.


### Parameters

In [120]:
testColumn = 'Item'

In [121]:
# user input this shit 

def presence_checker(df, column):
    """
    Checks for missing values in a specified column.
    
    Parameters:
    df : DataFrame
    column : str, column name to check for missing values.
    
    Returns:
    DataFrame with rows where the specified column is missing.
    """
    missing = df[df[column].isna() | (df[column].str.lower() == 'unknown')]
    
    if missing.empty:
        print(f"No missing values found in '{column}'.")
    else:
        print(f"Found {len(missing)} missing values in '{column}'.")
        print(missing[[column]].head())  # Display first 2 rows with missing values
    
    return missing


# Example usage:
missing_values = presence_checker(cafeSet, testColumn)

Found 677 missing values in 'Item'.
       Item
6   UNKNOWN
8       NaN
30      NaN
31  UNKNOWN
33  UNKNOWN


## Length Errors

A Length Check ensures that the appropriate number of characters are entered into the field. It verifies that the entered character string is neither too short nor too long. Consider a password that must be at least 8 characters long. The Length Check ensures that the field is filled with exactly 8 characters.

### Parameters

In [122]:
testColumn = 'Price Per Unit'
testLength = 3

In [123]:
#again above 

def length_checker(df, column, length):
    """
    Checks for length errors in a specified column.
    
    Parameters:
    df : DataFrame
    column : str, column name to check.
    length : int, required length for each value.
    
    Returns:
    DataFrame with rows where the specified column does not meet the length requirement.
    """
    # Convert column to string and check length
    invalid_length = df[df[column].astype(str).str.len() != length]
    
    if invalid_length.empty:
        print(f"All values in '{column}' meet the length requirement of {length}.")
    else:
        print(f"Found {len(invalid_length)} entries in '{column}' that do not meet the length requirement of {length}.")
        print(invalid_length[[column]].head(2))  # Display first 2 invalid entries
    
    return invalid_length

# Example usage:
invalid_length = length_checker(cafeSet, testColumn, testLength)

Found 354 entries in 'Price Per Unit' that do not meet the length requirement of 3.
    Price Per Unit
68           ERROR
140        UNKNOWN


## Look-Up Errors

Look Up assists in reducing errors in a field with a limited set of values. It consults a table to find acceptable values. The fact that there are only 7 possible days in a week, for example, ensures that the list of possible values is limited


### Parameters

In [124]:
testColumn = 'Payment Method'
valid_payment_methods = ["Credit Card", "Cash", "Digital Wallet"]

In [125]:
def lookup_checker(df, column, valid_values):
    """
    Checks for look-up errors in a specified column based on a list of valid values.
    
    Parameters:
    df : DataFrame
    column : str, column name to check.
    valid_values : list, list of expected values.
    
    Returns:
    DataFrame with rows where the column contains invalid values.
    """
    invalid_values = df[~df[column].isin(valid_values)]
    
    if invalid_values.empty:
        print(f"All values in '{column}' are valid.")
    else:
        print(f"Found {len(invalid_values)} invalid entries in '{column}'.")
        print(invalid_values[[column]].head(2))  # Display first 2 invalid entries
    
    return invalid_values


# Check Payment Method in cafeSet
lookup_errors = lookup_checker(cafeSet, testColumn, valid_payment_methods)


Found 3178 invalid entries in 'Payment Method'.
  Payment Method
3        UNKNOWN
6          ERROR


## Exact Duplicate Errors

## Near Duplicate Errors